In [88]:
from bs4 import BeautifulSoup
import requests
import datetime as dt
from urllib.parse import urljoin
import time
import json
import re
import os 

In [100]:
# HUMOR_URL = 'https://web.humoruniv.com/board/humor/list.html?table=pds'     # 웃긴자료
# HUMOR_URL = 'https://web.humoruniv.com/board/humor/list.html?table=fear'    # 공포
HUMOR_URL = 'https://web.humoruniv.com/board/humor/list.html?table=pdswait' #대기자료
HUMOR_HEADERS = { 'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36'}

In [101]:
import datetime as dt
import requests
from bs4 import BeautifulSoup

def fetch_posts_by_date_range(start_date, end_date):
    links = []
    current_page = 0
    continue_search = True

    # 날짜 형식을 datetime 객체로 변환
    start_date = dt.datetime.strptime(start_date, "%Y-%m-%d").date()
    end_date = dt.datetime.strptime(end_date, "%Y-%m-%d").date()

    while continue_search:
#         print(f"\nFetching page {current_page + 1}...")  # 페이지 진행상황
        url = f"{HUMOR_URL}&pg={current_page}"
        resp = requests.get(url, headers=HUMOR_HEADERS)
        resp.raise_for_status()
        resp.encoding = 'euc-kr'
        
        html = resp.text
        soup = BeautifulSoup(html, 'html.parser')

        # 각 게시물의 정보를 추출합니다.
        rows = soup.find_all('tr', id=lambda x: x and x.startswith('li_chk'))
        found_links_this_page = 0
        reached_earlier_date = False  # start_date 이전 날짜를 찾았는지 여부
        
        for row in rows:
            link_tag = row.find('a', href=True)
            date_tag = row.find('td', class_='li_date')
            if link_tag and date_tag:
                date = date_tag.find('span', class_='w_date').text.strip()
                time = date_tag.find('span', class_='w_time').text.strip()
                
                if date and time:  # 날짜와 시간이 비어있지 않은 경우에만 처리
                    full_date = f"{date} {time}"
                    try:
                        post_date = dt.datetime.strptime(full_date, "%Y-%m-%d %H:%M").date()

                        if start_date <= post_date <= end_date:
                            link = 'https://web.humoruniv.com/board/humor/' + link_tag['href']
                            found_links_this_page += 1
                            links.append(link)
                        elif post_date < start_date:
                            reached_earlier_date = True  # start_date보다 이전 날짜 발견
                            break  # 조건에 맞지 않으므로 해당 페이지를 건너뛰기
                    except ValueError:
                        continue  # Date parsing error 무시하고 다음으로 진행
        print(f"Processed page {current_page+1}, total links collected: {len(links)}")
        
        # 페이지 처리 후 continue_search 설정
        if reached_earlier_date:
            print("Reached an earlier date than the start_date. Exiting search.")
            continue_search = False
        elif found_links_this_page == 0:
            print("No more posts found in date range on this page. Exiting.")
            current_page += 1
            #             break
        else:
            print(f"Page {current_page + 1} processed: Found {found_links_this_page} links.")
            current_page += 1  # 다음 페이지로 이동

    print(f"\nTotal links found: {len(links)}")
    return links


In [118]:
def get_humoruniv_content(link):
    resp = requests.get(link, headers = HUMOR_HEADERS)
    resp.raise_for_status()
    resp.encoding = 'euc-kr'
    html = resp.text
    soup = BeautifulSoup(html, 'html.parser')

    post_id = re.search(r'number=(\d+)', link).group(1) if re.search(r'number=(\d+)', link) else None
#     post_id = soup.find('div', id='content_info').find_all('span')[0].text.strip()
    community = 'HUMORUIV'
#     category = soup.find('div', {'class':'ct_t'}).text.strip()
    category = soup.find('a', {'class':'bo'}).text.strip()
    title = soup.find('span', id='ai_cm_title').text.strip()
    vote_up = soup.find('span', id='ok_div').text.strip()
#     vote_down = soup.find('span', id='not_ok_span').text.strip()
    vote_down_tag = soup.find('span', id='not_ok_span')
    vote_down = vote_down_tag.text.strip() if vote_down_tag else "0"
    view = soup.find('div', id='content_info').find(string=re.compile('조회')).find_next('span').text.strip()
    view = view.replace(',', '')  # 쉼표 제거
    writer = soup.find('span', class_='hu_nick_txt').text.strip()
    date = soup.find('div', id='content_info').find(string=re.compile('작성시간')).find_next('span').text.strip()
    
    content = soup.find('span', id='ai_cm_content').get_text(separator="\n", strip=True)
    
    comments = []
    comments_section = soup.select('span.cmt_list:not(#cmt_best_comm_table span.cmt_list), span.cmt_text:not(#cmt_best_comm_table span.cmt_text)')
    for comment in comments_section:
        comment_text = comment.get_text(strip=True)
        comments.append(comment_text)

    data = {
        'id': 'HUMORUIV' + post_id,
        'community': community,
        'link': link,
        'category': category,
        'title': title,
        'vote_up': vote_up,
        'vote_down': vote_down,
        'view': view,
        'writer': writer,
        'date': date,
        'content': content,
        'comments': comments,
        'ncomment' : len(comments),
        'timestamp': dt.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    }
#     time.sleep(0.5)
    return data #title, date, writer, count, content.strip()

In [103]:
def retry_failed_links(failed_file, output_file, existing_ids, all_content):
    try:
        # 실패한 링크 파일 열기
        with open(failed_file, "r", encoding="utf-8") as f:
            failed_links = json.load(f)
        
        # 재시도 실패한 링크 저장용
        failed_links_retry = []
        
        for idx, link in enumerate(failed_links, start=1):
            try:
                content = get_instiz_content(link)
                if content['id'] not in existing_ids:  # 중복 확인
                    all_content.append(content)
                    existing_ids.add(content['id'])  # 중복 방지를 위해 추가된 ID를 기록
                    print(f"[Retry {idx}/{len(failed_links)}] Successfully processed: {link}")
                else:
                    print(f"[Retry {idx}/{len(failed_links)}] Duplicate ID, skipped: {content['id']}")
                time.sleep(1)
            except Exception as e:
                print(f"[Retry {idx}/{len(failed_links)}] Error occurred while processing {link}: {e}")
                failed_links_retry.append(link)  # 재시도 실패한 링크 저장

        # 모든 데이터 처리 후 저장
        with open(output_file, "w", encoding="utf-8") as f:
            json.dump(all_content, f, indent=4, ensure_ascii=False)
        print(f"\nAll records saved to {output_file} (Total records: {len(all_content)})")

        # 재시도 실패한 링크 저장
        if failed_links_retry:
            with open("failed_links_retry.json", "w", encoding="utf-8") as f:
                json.dump(failed_links_retry, f, indent=4, ensure_ascii=False)
            print(f"Retry failed links saved to failed_links_retry.json (Total failed: {len(failed_links_retry)})")
    except FileNotFoundError:
        print(f"No failed links file found: {failed_file}")

In [104]:
# filename = "HUMOR_CONTENTS.json"

# # 파일 내용을 비웁니다
# with open(filename, "w") as file:
#     file.write("")

In [124]:
filename = "HUMOR_CONTENTS.json"
existing_ids = set()
all_content = []

if os.path.exists(filename):
    with open(filename, "r", encoding="utf-8") as f:
        try:
            existing_content = json.load(f)
            existing_ids = {item['id'] for item in existing_content}  # 기존 JSON 파일의 post_id 수집
            all_content.extend(existing_content)  # 기존 데이터를 리스트에 추가
        except json.JSONDecodeError:
            pass  # 파일이 비어있거나 포맷이 잘못된 경우 무시하고 진행

In [107]:
start_date = "2024-10-15"
end_date = "2024-11-16"

info = fetch_posts_by_date_range(start_date, end_date)
failed_links = []  # 에러가 발생한 링크를 저장할 리스트
total_links = len(info)

Processed page 1, total links collected: 20
Page 1 processed: Found 20 links.
Processed page 2, total links collected: 40
Page 2 processed: Found 20 links.
Processed page 3, total links collected: 60
Page 3 processed: Found 20 links.
Processed page 4, total links collected: 80
Page 4 processed: Found 20 links.
Processed page 5, total links collected: 100
Page 5 processed: Found 20 links.
Processed page 6, total links collected: 120
Page 6 processed: Found 20 links.
Processed page 7, total links collected: 140
Page 7 processed: Found 20 links.
Processed page 8, total links collected: 160
Page 8 processed: Found 20 links.
Processed page 9, total links collected: 180
Page 9 processed: Found 20 links.
Processed page 10, total links collected: 200
Page 10 processed: Found 20 links.
Processed page 11, total links collected: 220
Page 11 processed: Found 20 links.
Processed page 12, total links collected: 240
Page 12 processed: Found 20 links.
Processed page 13, total links collected: 260
Page

Processed page 102, total links collected: 2040
Page 102 processed: Found 20 links.
Processed page 103, total links collected: 2060
Page 103 processed: Found 20 links.
Processed page 104, total links collected: 2080
Page 104 processed: Found 20 links.
Processed page 105, total links collected: 2100
Page 105 processed: Found 20 links.
Processed page 106, total links collected: 2120
Page 106 processed: Found 20 links.
Processed page 107, total links collected: 2140
Page 107 processed: Found 20 links.
Processed page 108, total links collected: 2160
Page 108 processed: Found 20 links.
Processed page 109, total links collected: 2180
Page 109 processed: Found 20 links.
Processed page 110, total links collected: 2200
Page 110 processed: Found 20 links.
Processed page 111, total links collected: 2220
Page 111 processed: Found 20 links.
Processed page 112, total links collected: 2240
Page 112 processed: Found 20 links.
Processed page 113, total links collected: 2260
Page 113 processed: Found 20

Processed page 200, total links collected: 4000
Page 200 processed: Found 20 links.
Processed page 201, total links collected: 4020
Page 201 processed: Found 20 links.
Processed page 202, total links collected: 4040
Page 202 processed: Found 20 links.
Processed page 203, total links collected: 4060
Page 203 processed: Found 20 links.
Processed page 204, total links collected: 4080
Page 204 processed: Found 20 links.
Processed page 205, total links collected: 4100
Page 205 processed: Found 20 links.
Processed page 206, total links collected: 4120
Page 206 processed: Found 20 links.
Processed page 207, total links collected: 4140
Page 207 processed: Found 20 links.
Processed page 208, total links collected: 4160
Page 208 processed: Found 20 links.
Processed page 209, total links collected: 4180
Page 209 processed: Found 20 links.
Processed page 210, total links collected: 4200
Page 210 processed: Found 20 links.
Processed page 211, total links collected: 4220
Page 211 processed: Found 20

Processed page 299, total links collected: 5980
Page 299 processed: Found 20 links.
Processed page 300, total links collected: 6000
Page 300 processed: Found 20 links.
Processed page 301, total links collected: 6020
Page 301 processed: Found 20 links.
Processed page 302, total links collected: 6040
Page 302 processed: Found 20 links.
Processed page 303, total links collected: 6060
Page 303 processed: Found 20 links.
Processed page 304, total links collected: 6080
Page 304 processed: Found 20 links.
Processed page 305, total links collected: 6100
Page 305 processed: Found 20 links.
Processed page 306, total links collected: 6120
Page 306 processed: Found 20 links.
Processed page 307, total links collected: 6140
Page 307 processed: Found 20 links.
Processed page 308, total links collected: 6160
Page 308 processed: Found 20 links.
Processed page 309, total links collected: 6180
Page 309 processed: Found 20 links.
Processed page 310, total links collected: 6200
Page 310 processed: Found 20

Processed page 397, total links collected: 7940
Page 397 processed: Found 20 links.
Processed page 398, total links collected: 7960
Page 398 processed: Found 20 links.
Processed page 399, total links collected: 7980
Page 399 processed: Found 20 links.
Processed page 400, total links collected: 8000
Page 400 processed: Found 20 links.
Processed page 401, total links collected: 8020
Page 401 processed: Found 20 links.
Processed page 402, total links collected: 8040
Page 402 processed: Found 20 links.
Processed page 403, total links collected: 8060
Page 403 processed: Found 20 links.
Processed page 404, total links collected: 8080
Page 404 processed: Found 20 links.
Processed page 405, total links collected: 8100
Page 405 processed: Found 20 links.
Processed page 406, total links collected: 8120
Page 406 processed: Found 20 links.
Processed page 407, total links collected: 8140
Page 407 processed: Found 20 links.
Processed page 408, total links collected: 8160
Page 408 processed: Found 20

...

Processed page 593, total links collected: 11860
Page 593 processed: Found 20 links.
Processed page 594, total links collected: 11880
Page 594 processed: Found 20 links.
Processed page 595, total links collected: 11900
Page 595 processed: Found 20 links.
Processed page 596, total links collected: 11920
Page 596 processed: Found 20 links.
Processed page 597, total links collected: 11940
Page 597 processed: Found 20 links.
Processed page 598, total links collected: 11960
Page 598 processed: Found 20 links.
Processed page 599, total links collected: 11980
Page 599 processed: Found 20 links.
Processed page 600, total links collected: 12000
Page 600 processed: Found 20 links.
Processed page 601, total links collected: 12020
Page 601 processed: Found 20 links.
Processed page 602, total links collected: 12040
Page 602 processed: Found 20 links.
Processed page 603, total links collected: 12060
Page 603 processed: Found 20 links.
Processed page 604, total links collected: 12080
Page 604 process

Processed page 691, total links collected: 13820
Page 691 processed: Found 20 links.
Processed page 692, total links collected: 13840
Page 692 processed: Found 20 links.
Processed page 693, total links collected: 13860
Page 693 processed: Found 20 links.
Processed page 694, total links collected: 13880
Page 694 processed: Found 20 links.
Processed page 695, total links collected: 13900
Page 695 processed: Found 20 links.
Processed page 696, total links collected: 13920
Page 696 processed: Found 20 links.
Processed page 697, total links collected: 13940
Page 697 processed: Found 20 links.
Processed page 698, total links collected: 13960
Page 698 processed: Found 20 links.
Processed page 699, total links collected: 13980
Page 699 processed: Found 20 links.
Processed page 700, total links collected: 14000
Page 700 processed: Found 20 links.
Processed page 701, total links collected: 14020
Page 701 processed: Found 20 links.
Processed page 702, total links collected: 14040
Page 702 process

Processed page 789, total links collected: 15780
Page 789 processed: Found 20 links.
Processed page 790, total links collected: 15800
Page 790 processed: Found 20 links.
Processed page 791, total links collected: 15820
Page 791 processed: Found 20 links.
Processed page 792, total links collected: 15840
Page 792 processed: Found 20 links.
Processed page 793, total links collected: 15860
Page 793 processed: Found 20 links.
Processed page 794, total links collected: 15880
Page 794 processed: Found 20 links.
Processed page 795, total links collected: 15900
Page 795 processed: Found 20 links.
Processed page 796, total links collected: 15920
Page 796 processed: Found 20 links.
Processed page 797, total links collected: 15940
Page 797 processed: Found 20 links.
Processed page 798, total links collected: 15960
Page 798 processed: Found 20 links.
Processed page 799, total links collected: 15980
Page 799 processed: Found 20 links.
Processed page 800, total links collected: 16000
Page 800 process

Processed page 886, total links collected: 17720
Page 886 processed: Found 20 links.
Processed page 887, total links collected: 17740
Page 887 processed: Found 20 links.
Processed page 888, total links collected: 17760
Page 888 processed: Found 20 links.
Processed page 889, total links collected: 17780
Page 889 processed: Found 20 links.
Processed page 890, total links collected: 17800
Page 890 processed: Found 20 links.
Processed page 891, total links collected: 17820
Page 891 processed: Found 20 links.
Processed page 892, total links collected: 17840
Page 892 processed: Found 20 links.
Processed page 893, total links collected: 17860
Page 893 processed: Found 20 links.
Processed page 894, total links collected: 17880
Page 894 processed: Found 20 links.
Processed page 895, total links collected: 17900
Page 895 processed: Found 20 links.
Processed page 896, total links collected: 17920
Page 896 processed: Found 20 links.
Processed page 897, total links collected: 17940
Page 897 process

Processed page 984, total links collected: 19680
Page 984 processed: Found 20 links.
Processed page 985, total links collected: 19700
Page 985 processed: Found 20 links.
Processed page 986, total links collected: 19720
Page 986 processed: Found 20 links.
Processed page 987, total links collected: 19740
Page 987 processed: Found 20 links.
Processed page 988, total links collected: 19760
Page 988 processed: Found 20 links.
Processed page 989, total links collected: 19780
Page 989 processed: Found 20 links.
Processed page 990, total links collected: 19800
Page 990 processed: Found 20 links.
Processed page 991, total links collected: 19820
Page 991 processed: Found 20 links.
Processed page 992, total links collected: 19840
Page 992 processed: Found 20 links.
Processed page 993, total links collected: 19860
Page 993 processed: Found 20 links.
Processed page 994, total links collected: 19880
Page 994 processed: Found 20 links.
Processed page 995, total links collected: 19900
Page 995 process

In [110]:
a = 'https://web.humoruniv.com/board/humor/read.html?table=pdswait&pg=140&number=11333098'
get_humoruniv_content(a)

{'id': 'HUMORUIV11333098',
 'community': 'HUMORUIV',
 'link': 'https://web.humoruniv.com/board/humor/read.html?table=pdswait&pg=140&number=11333098',
 'category': '대기자료',
 'title': '팔아야 한다 vs 유지해야 한다.',
 'vote_up': '0',
 'vote_down': '0',
 'view': '248',
 'writer': '폴리페놀1420',
 'date': '2024-11-14 01:53:52',
 'content': '어떡하지',
 'comments': ['와 개쩌네',
  '이거 이렇게나 많이 오를거라곤 상상도 못했어',
  '치킨 사줘',
  '무슨 맛 사줄까',
  '팔아서나한푼만',
  '하앙아 오빠라고 불러보렴',
  '오라방 :3!!',
  '오..오..아저씨...!!',
  '��',
  '오빠 저는 교촌 옥수수 순살이요!',
  '우리집으로 와',
  '9800 주 샀으면 3 억 7 천만인데 ㄲㅂ,,',
  '제 3억 7천만원 돌려주세요'],
 'ncomment': 13,
 'timestamp': '2024-11-16 10:40:13'}

In [133]:
# 새 데이터를 가져오고 기존 ID와 비교 후 추가
for idx, link in enumerate(info[20019:], start=20019):
    try:
        content = get_humoruniv_content(link)
        if content['id'] not in existing_ids:  # 중복 확인
            all_content.append(content)
            existing_ids.add(content['id'])  # 중복 방지를 위해 추가된 ID를 기록
            print(f"[{idx}/{total_links}] Successfully processed: {link}")
        else:
            print(f"[{idx}/{total_links}] Duplicate ID, skipped: {content['id']}")
    except Exception as e:
        print(f"[{idx}/{total_links}] Error occurred while processing {link}: {e}")
        failed_links.append(link)  # 에러난 링크 저장

    if idx >= total_links:
        break
# 모든 데이터 처리 후 한 번만 저장
with open(filename, "w", encoding="utf-8") as f:
    json.dump(all_content, f, indent=4, ensure_ascii=False)
print(f"\nAll records saved to {filename} (Total records: {len(all_content)})")

# 실패한 링크 저장
if failed_links:
    with open("failed_links.json", "w", encoding="utf-8") as f:
        json.dump(failed_links, f, indent=4, ensure_ascii=False)
    print(f"Failed links saved to failed_links.json (Total failed: {len(failed_links)})")

print("json end")

[20019/20020] Duplicate ID, skipped: HUMORUIV11306933

All records saved to HUMOR_CONTENTS.json (Total records: 24467)
Failed links saved to failed_links.json (Total failed: 9712)
json end
